<a href="https://colab.research.google.com/github/minizhiren/498-hw2/blob/main/mp4test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# STEP 0: 安装依赖
!pip install -q datasets transformers peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [3]:
# STEP 1: 加载 StableCode 3B 模型和 Tokenizer（支持 LoRA）
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, prepare_model_for_kbit_training, LoraConfig, TaskType
import torch

model_id = "stabilityai/stable-code-3b"

# 加载 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token  # 关键修复


# 4-bit 量化配置
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,  # 注意：需要支持 bfloat16 的 GPU
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# 加载模型（支持量化 + 自定义架构）
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# 准备 LoRA 微调：冻结原始参数，只训练 adapter
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # 对于代码模型，这两个通常有效
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/602 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/610M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
# -----------------------------------------------
# STEP 2 – 载入数据（你的 json 必须含 prompt / answer / explanation 字段）
# -----------------------------------------------
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

train_json_path = "100.json"
train_ds = load_dataset("json", data_files=train_json_path)["train"]

print("Columns:", train_ds.column_names)  # ▶ ['prompt', 'answer', 'explanation']

# -----------------------------------------------
# STEP 3 – Tokenize (仅对 answer 计算损失)

END = "<|stop|>"          # 终止标记，写在 answer 里
PROMPT_SUFFIX = "Answer:" # 明确提示词

def format_example(ex, max_len=512):
    # ① prompt 末尾直接加 “Answer:”（带一个空格更自然）
    prompt_txt  = ex["prompt"].rstrip()
    # ② answer 末尾加 <|stop|> + eos
    answer_txt  = ex["answer"].rstrip() + f" {END}" + tokenizer.eos_token

    # ③ 编码
    prompt_ids  = tokenizer(prompt_txt,  add_special_tokens=False).input_ids
    answer_ids  = tokenizer(answer_txt, add_special_tokens=False).input_ids

    # ④ 拼接；labels 对 prompt（含 “Answer:”）全部 -100
    input_ids   = prompt_ids + answer_ids
    attention   = [1] * len(input_ids)
    labels      = [-100] * len(prompt_ids) + answer_ids

    # ⑤ pad / truncate
    pad = max_len - len(input_ids)
    if pad < 0:
        input_ids, attention, labels = (
            seq[:max_len] for seq in (input_ids, attention, labels)
        )
    else:
        input_ids += [tokenizer.pad_token_id] * pad
        attention += [0] * pad
        labels    += [-100] * pad

    return {"input_ids": input_ids,
            "attention_mask": attention,
            "labels": labels}

# ⚡ tokenized_ds
tokenized_ds = train_ds.map(format_example,
                            remove_columns=train_ds.column_names,  # 把 prompt/answer/explanation都移除
                            num_proc=2)

print("Tokenized sample:", tokenized_ds[0].keys())

# 与 causal LM 搭配的默认 DataCollator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# -----------------------------------------------
# STEP 4 – Trainer 超参
# -----------------------------------------------
output_dir = "stable-code-3b-lora"
args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    fp16=False,
    bf16=True,
    logging_strategy="steps",
    logging_steps=10,
    disable_tqdm=True,
    learning_rate=1e-4,
    save_strategy="epoch",
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="none",
    remove_unused_columns=False
)

# -----------------------------------------------
# STEP 5 – 训练
# -----------------------------------------------
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=data_collator
)
model.config.use_cache = False
trainer.train()
trainer.save_model(output_dir)# 存 LoRA
tokenizer.save_pretrained(output_dir)



Generating train split: 0 examples [00:00, ? examples/s]

Columns: ['prompt', 'answer']


Map (num_proc=2):   0%|          | 0/1218 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Tokenized sample: dict_keys(['input_ids', 'attention_mask', 'labels'])


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


{'loss': 2.8643, 'grad_norm': 1.3951627016067505, 'learning_rate': 9.998810135399546e-05, 'epoch': 0.13114754098360656}
{'loss': 2.2345, 'grad_norm': 1.4734300374984741, 'learning_rate': 9.856710349066307e-05, 'epoch': 0.26229508196721313}
{'loss': 1.3576, 'grad_norm': 1.8339790105819702, 'learning_rate': 9.484363707663442e-05, 'epoch': 0.39344262295081966}
{'loss': 0.7853, 'grad_norm': 1.3218621015548706, 'learning_rate': 8.899422415464409e-05, 'epoch': 0.5245901639344263}
{'loss': 0.5689, 'grad_norm': 1.9595361948013306, 'learning_rate': 8.129617360920296e-05, 'epoch': 0.6557377049180327}
{'loss': 0.47, 'grad_norm': 1.2092068195343018, 'learning_rate': 7.211443451095007e-05, 'epoch': 0.7868852459016393}
{'loss': 0.3655, 'grad_norm': 1.3519694805145264, 'learning_rate': 6.188429461630866e-05, 'epoch': 0.9180327868852459}


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


{'loss': 0.3096, 'grad_norm': 1.215887427330017, 'learning_rate': 5.1090744251728064e-05, 'epoch': 1.039344262295082}
{'loss': 0.2705, 'grad_norm': 1.7792712450027466, 'learning_rate': 4.0245483899193595e-05, 'epoch': 1.1704918032786886}
{'loss': 0.2471, 'grad_norm': 1.451256275177002, 'learning_rate': 2.9862665507063147e-05, 'epoch': 1.301639344262295}
{'loss': 0.2415, 'grad_norm': 1.020472526550293, 'learning_rate': 2.0434517581820896e-05, 'epoch': 1.4327868852459016}
{'loss': 0.2233, 'grad_norm': 1.1068311929702759, 'learning_rate': 1.2408009626051137e-05, 'epoch': 1.5639344262295083}
{'loss': 0.2359, 'grad_norm': 1.418294072151184, 'learning_rate': 6.163662214624616e-06, 'epoch': 1.6950819672131148}
{'loss': 0.2213, 'grad_norm': 1.1273175477981567, 'learning_rate': 1.99750728070357e-06, 'epoch': 1.8262295081967213}
{'loss': 0.2093, 'grad_norm': 1.1205263137817383, 'learning_rate': 1.0705383806982606e-07, 'epoch': 1.957377049180328}
{'train_runtime': 283.2784, 'train_samples_per_sec

('stable-code-3b-lora/tokenizer_config.json',
 'stable-code-3b-lora/special_tokens_map.json',
 'stable-code-3b-lora/tokenizer.json')

In [7]:
import shutil
output_dir = "/content/stable-code-3b-lora"          # 你的模型目录
zip_name   = "stable-code-3b-lora"          # 生成 stable-code-3b-lora.zip

# 第三个参数 root_dir 指向要打包的目录
# 如果不想把绝对路径写进 zip，可把 base_dir 指向目录名
shutil.make_archive(zip_name, "zip", root_dir=output_dir, base_dir=".")
print(f"{zip_name}.zip 已生成")

stable-code-3b-lora.zip 已生成


In [8]:
from google.colab import files
files.download('stable-code-3b-lora.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# ============================================================
# STEP 6 – 载入 LoRA & 组装推理模型
# ============================================================
import torch, pandas as pd
from datasets           import load_dataset
from transformers        import AutoTokenizer, AutoModelForCausalLM
from peft               import PeftModel

# ---------- 0. 目录 & 常量 ----------
output_dir = "stable-code-3b-lora"     # 训练阶段保存的目录
model_id   = "stabilityai/stable-code-3b"  # 你的基础模型

# ---------- 1. 加载 tokenizer（保证含 SEP / END） ----------
tokenizer = AutoTokenizer.from_pretrained(output_dir)
tokenizer.pad_token    = tokenizer.eos_token
tokenizer.padding_side = "left"

# ---------- 2. 加载基座 + LoRA 适配器 ----------
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,    # ← 训练脚本里同一个 bnb_config
    device_map="auto"
)

inference_model = PeftModel.from_pretrained(base_model, output_dir).eval()

# ============================================================
# STEP 7 – batch inference → CSV  (clean “Answer:” prefixes)
# ============================================================
import re, torch, pandas as pd
from datasets import load_dataset
from transformers import StoppingCriteria, StoppingCriteriaList

PROMPT_SUFFIX = "Answer:"          # prompt ends with this token + space
STOP_STR      = "<|stop|>"         # appears at end of every training answer
BS            = 8                  # batch size
MAX_NEW       = 64                 # generation upper-bound (tune if needed)

# ---------- 1. test set ----------
test_ds = load_dataset("json", data_files="prompt.json")["train"]
print("Test size:", len(test_ds))

# ---------- 2. ensure <|stop|> in vocab ----------
if STOP_STR not in tokenizer.get_vocab():
    tokenizer.add_tokens([STOP_STR])
stop_id = tokenizer.convert_tokens_to_ids(STOP_STR)

# ---------- 3. (optional) block byte & non-ASCII ----------
byte_ids = [i for t,i in tokenizer.get_vocab().items()
            if t.startswith("<0x") and t.endswith(">")]
non_ascii = [i for t,i in tokenizer.get_vocab().items()
             if any(ord(ch) > 127 for ch in t) and i not in byte_ids]
bad_words_ids = [[i] for i in set(byte_ids+non_ascii)]

# ---------- 4. stopper ----------
class StopOnId(StoppingCriteria):
    def __init__(self, tid): self.tid = tid
    def __call__(self, ids, *_): return ids[0, -1] == self.tid
stoppers = StoppingCriteriaList([StopOnId(stop_id)])

# ---------- 5. prompt helper ----------
def build_prompt(q:str) -> str:
    """append exactly one 'Answer: '"""
    q = q.rstrip()

    return q                     # trailing space for nicer spacing

# ---------- 6. batch generation ----------
@torch.inference_mode()
def batch_generate(prompts):
    enc = tokenizer(prompts, padding=True, truncation=True,
                    max_length=2048, return_tensors="pt").to("cuda")
    p_lens = (enc.input_ids != tokenizer.pad_token_id).sum(1)

    outs = inference_model.generate(
        **enc,
        max_new_tokens=MAX_NEW,
        do_sample=False,
        eos_token_id=stop_id,           # stop when <|stop|> arrives
        stopping_criteria=stoppers,
        pad_token_id=tokenizer.eos_token_id,
        bad_words_ids=bad_words_ids
    )

    cleaned = []
    for i,l in enumerate(p_lens):
        txt = tokenizer.decode(outs[i, l:], skip_special_tokens=True)
        txt = txt.split(STOP_STR, 1)[0]                 # cut at <|stop|>
        # remove every leading "Answer:" / "answer:" (possibly repeated)
        txt = re.sub(r'^(?:Answer[:\s]*)+', '', txt, flags=re.I).strip()
        # OPTIONAL: keep only a one-line python-style statement
        m = re.search(r'^[A-Za-z_]\w*(?:\.\w+)*\(.*\)$', txt)
        cleaned.append(m.group(0) if m else txt)
    return cleaned

# ---------- 7. iterate & collect ----------
records = []
for idx in range(0, len(test_ds), BS):
    batch_q  = test_ds[idx:idx+BS]["prompt"]
    outputs  = batch_generate([build_prompt(p) for p in batch_q])

    records.extend({"prompt": q, "completion": a}
                   for q, a in zip(batch_q, outputs))

    if idx % 32 == 0:
        print(f"{idx}/{len(test_ds)} processed…")

# ---------- 8. write CSV ----------
csv_path = "stable_code3b_lora_test_outputs.csv"
pd.DataFrame(records).to_csv(csv_path, index=False, encoding="utf-8")
print(f"✅ CSV saved → {csv_path}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Test size: 50
0/50 processed…
32/50 processed…
✅ CSV saved → stable_code3b_lora_test_outputs.csv


In [ ]:
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)

SyntaxError: invalid syntax (<ipython-input-9-eb230817b0bc>, line 1)

In [ ]:
!pip install -q pandas sentencepiece

Test size: 99
0/99 done …
32/99 done …
64/99 done …
96/99 done …
✅ CSV saved → stable_code3b_lora_test_outputs.csv


In [ ]:
# STEP 4: 设置训练参数并启动训练
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./qwen2.5_lora_output",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    fp16=False,  # 使用 bf16，所以 fp16 不开
    bf16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, padding=True)
)

trainer.train()

<ipython-input-10-614da48f827e>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*arg

Step,Training Loss
10,9.877500
20,0.492300
30,0.229400
40,0.131600
50,0.092400
60,0.054600
70,0.041400
80,0.028800
90,0.027100
100,0.025500


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=450, training_loss=0.2583870980805821, metrics={'train_runtime': 3605.1989, 'train_samples_per_second': 0.25, 'train_steps_per_second': 0.125, 'total_flos': 7380043038720000.0, 'train_loss': 0.2583870980805821, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./stablecode3b_lora_adapter")
tokenizer.save_pretrained("./stablecode3b_lora_adapter")


('./stablecode3b_lora_adapter/tokenizer_config.json',
 './stablecode3b_lora_adapter/special_tokens_map.json',
 './stablecode3b_lora_adapter/tokenizer.json')

In [ ]:
# 保存 LoRA adapter 和 tokenizer 到 Colab 文件系统
output_dir = "./stablecode3b_lora_adapter"

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

import shutil

shutil.make_archive("stablecode3b_lora_adapter", "zip", output_dir)


'/content/stablecode3b_lora_adapter.zip'

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

model_id = "stabilityai/stable-code-3b"
adapter_path = "./stablecode3b_lora_adapter"

# 加载基础模型
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16  # or bf16 if支持
)

# 加载微调的 LoRA adapter
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.eval()
tokenizer = AutoTokenizer.from_pretrained(adapter_path)
tokenizer.pad_token = tokenizer.eos_token


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def generate_response(prompt, model, tokenizer, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)
test_prompt = "How do you enable prefix caching in vllm?"
response = generate_response(test_prompt, model, tokenizer)
print("🤖 模型回答：\n", response)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


🤖 模型回答：
 How do you enable prefix caching in vllm?
    # prefix_cache_params = dict(
    #     max_size=2048,
    #     # This next line enables prefix caching. This is optional.
    #     enable_prefix_caching=True,
    #     # This next line specifies how to compute prefix hashes. This is
    #     # optional. If omitted, llama will hash the entire request
    #     # prefix.
    #     prefix_key_fn=lambda req: req.inputs[0][:64],
    # )
    # llama = EngineArgs(
